# Third Project: Building the Abstract Syntax Tree

Abstract syntax trees are data structures that better represent the structure of the
program code than the parse tree. An AST can be edited and enhanced with information
such as properties and annotations for each element it contains. Your goal in this
third project is to transform the parse tree into an AST.

## The main uc_compiler.py module

First, in order to standardize the way the compiler we are building is called, I am suggesting that you use the main compiler module
attached [here](./src/uc_compiler.py).

note: you can create stubs for the classes and methods in the uC module as a temporary
substitute for the code yet to be developed, or just comment on the parts of the code
corresponding to the future compilation phases.

## Abstract Syntax Tree objects
This section defines classes for different kinds of nodes of an Abstract Syntax Tree.
During parsing, you will create these nodes and connect them together. In general,
you will have a different AST node for each kind of grammar rule.

In [ ]:
class Node(object):
    """
    Base class example for the AST nodes.

    By default, instances of classes have a dictionary for attribute storage.
    This wastes space for objects having very few instance variables.
    The space consumption can become acute when creating large numbers of instances.

    The default can be overridden by defining __slots__ in a class definition.
    The __slots__ declaration takes a sequence of instance variables and reserves
    just enough space in each instance to hold a value for each variable.
    Space is saved because __dict__ is not created for each instance.
    """
    __slots__ = ()

    def children(self):
        """ A sequence of all children that are Nodes. """
        pass

For each of specific AST nodes, you need to add the appropriate ```__slots__```
specification that indicates what fields are to be stored:

In [ ]:
class Program(Node):
    __slots__ = ('gdecls', 'coord')

    def __init__(self, gdecls, coord=None):
        self.gdecls = gdecls
        self.coord = coord

    def children(self):
        nodelist = []
        for i, child in enumerate(self.gdecls or []):
            nodelist.append(("gdecls[%d]" % i, child))
        return tuple(nodelist)

    attr_names = ()

Just as another example, for a binary operator, you might store the operator, the left
expression, and the right expression like this:

In [ ]:
class BinaryOp(Node):
    __slots__ = ("op", "lvalue", "rvalue", "coord")

    def __init__(self, op, left, right, coord=None):
        self.op = op
        self.lvalue = left
        self.rvalue = right
        self.coord = coord

    def children(self):
        nodelist = []
        if self.lvalue is not None:
            nodelist.append(("lvalue", self.lvalue))
        if self.rvalue is not None:
            nodelist.append(("rvalue", self.rvalue))
        return tuple(nodelist)

    attr_names = ("op",)

For Constant objects, you might store the type and value, like this:

In [ ]:
class Constant(Node):
    __slots__ = ("type", "value", "coord")

    def __init__(self, type, value, coord=None):
        self.type = type
        self.value = value
        self.coord = coord

    def children(self):
        return ()

    attr_names = ("type", "value")

Suggestion: You should start simple and incrementally work your way up to building the
complete grammar.

## AST node classes

The list below defines the AST node classes and expected attribute names that must be
used in uCParser:

ArrayDecl ( ), ArrayRef ( ), Assert ( ), Assignment (op), BinaryOp (op),
Break ( ), Compound ( ), Constant (type, value), Decl (name), DeclList ( ),
EmptyStatement ( ), ExprList ( ), For ( ), FuncCall ( ), FuncDecl ( ),
FuncDef ( ), GlobalDecl ( ), ID (name), If ( ), InitList ( ), ParamList ( ),
Print ( ), Program ( ), Read ( ), Return ( ), Type (name), VarDecl ( ),
UnaryOp (op), While ( ).

### Showing the AST

Consider the previous uC program example:

In [ ]:
/* comment */
int j = 3;
int main () {
  int i = j;
  int k = 3;
  int p = 2 * j;
  assert p == 2 * i;
}

The dump of the AST for the example above looks like this:

In [ ]:
Program:
    GlobalDecl:
        Decl: ID(name=j)
            VarDecl:
                Type: int @ 2:1
            Constant: int, 3 @ 2:9
    FuncDef:
        Type: int @ 3:1
        Decl: ID(name=main)
            FuncDecl:
                VarDecl:
                    Type: int @ 3:1
        Compound: @ 3:13
            Decl: ID(name=i)
                VarDecl:
                    Type: int @ 4:3
                ID: j @ 4:11
            Decl: ID(name=k)
                VarDecl:
                    Type: int @ 5:3
                Constant: int, 3 @ 5:11
            Decl: ID(name=p)
                VarDecl:
                    Type: int @ 6:3
                BinaryOp: * @ 6:11
                    Constant: int, 2 @ 6:11
                    ID: j @ 6:15
            Assert: @ 7:3
                BinaryOp: == @ 7:10
                    ID: p @ 7:10
                    BinaryOp: * @ 7:15
                        Constant: int, 2 @ 7:15
                        ID: i @ 7:19

The methods to generate a textual representation of the AST nodes and print all its
attributes is showing below:

In [ ]:
def represent_node(obj, indent):
    def _repr(obj, indent, printed_set):
        """
        Get the representation of an object, with dedicated pprint-like format for lists.
        """
        if isinstance(obj, list):
            indent += 1
            sep = ",\n" + (" " * indent)
            final_sep = ",\n" + (" " * (indent - 1))
            return (
                "["
                + (sep.join((_repr(e, indent, printed_set) for e in obj)))
                + final_sep
                + "]"
            )
        elif isinstance(obj, Node):
            if obj in printed_set:
                return ""
            else:
                printed_set.add(obj)
            result = obj.__class__.__name__ + "("
            indent += len(obj.__class__.__name__) + 1
            attrs = []
            for name in obj.__slots__[:-1]:
                if name == "bind":
                    continue
                value = getattr(obj, name)
                value_str = _repr(value, indent + len(name) + 1, printed_set)
                attrs.append(name + "=" + value_str)
            sep = ",\n" + (" " * indent)
            final_sep = ",\n" + (" " * (indent - 1))
            result += sep.join(attrs)
            result += ")"
            return result
        elif isinstance(obj, str):
            return obj
        else:
            return ""

    # avoid infinite recursion with printed_set
    printed_set = set()
    return _repr(obj, indent, printed_set)


class Node:
    """Abstract base class for AST nodes."""

    __slots__ = "coord"
    attr_names = ()

    def __init__(self, coord=None):
        self.coord = coord

    def __repr__(self):
        """Generates a python representation of the current node"""
        return represent_node(self, 0)

    def children(self):
        """A sequence of all children that are Nodes"""
        pass

    def show(
        self,
        buf=sys.stdout,
        offset=0,
        attrnames=False,
        nodenames=False,
        showcoord=False,
        _my_node_name=None,
    ):
        """Pretty print the Node and all its attributes and children (recursively) to a buffer.
        buf:
            Open IO buffer into which the Node is printed.
        offset:
            Initial offset (amount of leading spaces)
        attrnames:
            True if you want to see the attribute names in name=value pairs. False to only see the values.
        nodenames:
            True if you want to see the actual node names within their parents.
        showcoord:
            Do you want the coordinates of each Node to be displayed.
        """
        lead = " " * offset
        if nodenames and _my_node_name is not None:
            buf.write(lead + self.__class__.__name__ + " <" + _my_node_name + ">: ")
            inner_offset = len(self.__class__.__name__ + " <" + _my_node_name + ">: ")
        else:
            buf.write(lead + self.__class__.__name__ + ":")
            inner_offset = len(self.__class__.__name__ + ":")

        if self.attr_names:
            if attrnames:
                nvlist = [
                    (n, represent_node(getattr(self, n), offset+inner_offset+1+len(n)+1))
                    for n in self.attr_names
                    if getattr(self, n) is not None
                ]
                attrstr = ", ".join("%s=%s" % nv for nv in nvlist)
            else:
                vlist = [getattr(self, n) for n in self.attr_names]
                attrstr = ", ".join(
                    represent_node(v, offset + inner_offset + 1) for v in vlist
                )
            buf.write(" " + attrstr)

        if showcoord:
            if self.coord and self.coord.line != 0:
                buf.write(" %s" % self.coord)
        buf.write("\n")

        for (child_name, child) in self.children():
            child.show(buf, offset + 4, attrnames, nodenames, showcoord, child_name)

### Dealing with line and column information in AST

In [ ]:
class Coord:
    """Coordinates of a syntactic element. Consists of:
    - Line number
    - (optional) column number, for the Lexer
    """

    __slots__ = ("line", "column")

    def __init__(self, line, column=None):
        self.line = line
        self.column = column

    def __str__(self):
        if self.line and self.column is not None:
            coord_str = "@ %s:%s" % (self.line, self.column)
        elif self.line:
            coord_str = "@ %s" % (self.line)
        else:
            coord_str = ""
        return coord_str

The Coord class above should be used to store (and show in the AST) the lines and
columns of the productions in the source code. To capture the coordinates for a
production ```p``` of the parser indexed with ```token_idx``` use the following code
in the UCParser class (the coordinate includes the ```lineno``` and the ```column```.
Both follow the semantics of the lex, starting at 1):

In [ ]:
    def _token_coord(self, p, token_idx):
        last_cr = p.lexer.lexer.lexdata.rfind('\n', 0, p.lexpos(token_idx))
        if last_cr < 0:
            last_cr = -1
        column = (p.lexpos(token_idx) - (last_cr))
        return Coord(p.lineno(token_idx), column)

## Additional implementation aspects

### Building Declarations

At uC, statements can come several in a line. For example:
```
  int x, y, z = 5;
```
However, for AST, we must divide them to separate the Decl nodes.
The code snippet below splits the declarations and always returns a list of Decl nodes,
even if it is a single element.

In [ ]:
    def _build_declarations(self, spec, decls):
        """ Builds a list of declarations all sharing the given specifiers.
        """
        declarations = []

        for decl in decls:
            assert decl['decl'] is not None
            declaration = Decl(
                    name=None,
                    type=decl['decl'],
                    init=decl.get('init'),
                    coord=decl['decl'].coord)

            fixed_decl = self._fix_decl_name_type(declaration, spec)
            declarations.append(fixed_decl)

        return declarations

Due to the order in which declarators are constructed, they have to be fixed in order to
look like a normal AST.

When a declaration arrives from syntax construction, it has these problems:
 * The innermost VarDecl has no type (because the basic type is only known at the
   uppermost declaration level)
 * The declaration has no variable name since that is saved in the innermost VarDecl

The method below fixes these problems:

In [ ]:
    def _fix_decl_name_type(self, decl, typename):
        """ Fixes a declaration. Modifies decl.
        """
        # Reach the underlying basic type
        type = decl
        while not isinstance(type, VarDecl):
            type = type.type

        decl.name = type.declname
        if not typename:
            # Functions default to returning int
            if not isinstance(decl.type, FuncDecl):
                self._parse_error("Missing type in declaration", decl.coord)
            type.type = Type('int', coord=decl.coord)
        else:
            type.type = Type(typename.name, coord=typename.coord)
            
        return decl

The AST for the statement: ```int x, y, z = 5;```look like this:
```
Program:
    GlobalDecl:
        Decl: ID(name=x)
            VarDecl:
                Type: int @ 1:1
        Decl: ID(name=y)
            VarDecl:
                Type: int @ 1:1
        Decl: ID(name=z)
            VarDecl:
                Type: int @ 1:1
            Constant: int, 5 @ 1:15
```

### Building Function Definitions

Declarations always come as lists (because they can be several in one line); therefore,
a design decision was to wrap the function definition into a list as well, to make the
return value of global_declaration homogeneous. On the other hand, we do not embed
a ```FuncDef``` object inside the ```GlobalDecl``` class. So we will have:

In [ ]:
    def p_program(self, p):
        """ program  : global_declaration_list
        """
        p[0] = Program(p[1])

    def p_global_declaration_list(self, p):
        """ global_declaration_list : global_declaration
                                    | global_declaration_list global_declaration
        """
        p[0] = [p[1]] if len(p) == 2 else p[1] + [p[2]]

    def p_global_declaration_1(self, p):
        """ global_declaration    : declaration
        """
        p[0] = GlobalDecl(p[1])    // GlobalDecl is a AST class

    def p_global_declaration_2(self, p):
        """ global_declaration    : function_definition
        """
        p[0] = p[1]      # Note that FuncDef was not embedded into a GlobalDecl

No exemplo:
```
int i, j;
int main() {}
```
A seguinte AST é produzida:
```
Program:
    GlobalDecl:
        Decl: ID(name=i)
            VarDecl:
                Type: int @ 1:1
        Decl: ID(name=j)
            VarDecl:
                Type: int @ 1:1
    FuncDef:
        Type: int @ 2:1
        Decl: ID(name=main)
            FuncDecl:
                VarDecl:
                    Type: int @ 2:1
        ...
 ```

### Type Modifier

A uC type consists of a basic type declaration, with a list of modifiers. For example:
```  
int c[5];
```
The basic declaration here is 'int c', and the array is the modifier.
Basic declarations are represented by VarDecl and the modifiers are **FuncDecl**
 and **ArrayDecl**.

The standard states that whenever a new modifier is parsed, it should be added to the
end of the list of modifiers. For example:

**Array Declarators**

In a declaration T D where D has the form D1 [constant_expression_opt] and the type of
the identifier in the declaration T D1 is "type-modifier T", the type of the identifier
of D is "type-modifier array of T"

This is what the method below does. The declarator it receives can be a list of
declarators ending with VarDecl. It tacks the modifier to the end of this list, just
before the VarDecl.

In [ ]:
    def _type_modify_decl(self, decl, modifier):
        """Tacks a type modifier on a declarator, and returns
           the modified declarator.
           Note: the declarator and modifier may be modified
        """
        modifier_head = modifier
        modifier_tail = modifier

        # The modifier may be a nested list. Reach its tail.
        while modifier_tail.type:
            modifier_tail = modifier_tail.type

        # If the decl is a basic type, just tack the modifier onto it
        if isinstance(decl, VarDecl):
            modifier_tail.type = decl
            return modifier
        else:
            # Otherwise, the decl is a list of modifiers. Reach
            # its tail and splice the modifier onto the tail,
            # pointing to the underlying basic type.
            decl_tail = decl

            while not isinstance(decl_tail.type, VarDecl):
                decl_tail = decl_tail.type

            modifier_tail.type = decl_tail.type
            decl_tail.type = modifier_head
            return decl

The AST for the statement ```int c[5];``` look like this:

```
Program:
    GlobalDecl:
        Decl: ID(name=c)
            ArrayDecl:
                VarDecl:
                    Type: int @ 1:1
                Constant: int, 5 @ 1:7
```

### Single Expressions

Expressions in the uC grammar can be seen as lists but can also appear as single
expression. In the latter case, in order not to "pollute" AST, a design decision was to
treat single expression differently. See the example for the print statement below:

In [ ]:
char mc[] = "Susy";

int main(){
    print("Hello World!");
    print("Hello", mc, ". Welcome to MC921");
    return;
}

Program:
    GlobalDecl:
        Decl: ID(name=mc)
            ArrayDecl:
                VarDecl:
                    Type: char @ 1:1
            Constant: string, Susy @ 1:13
    FuncDef:
        Type: int @ 3:1
        Decl: ID(name=main)
            FuncDecl:
                VarDecl:
                    Type: int @ 3:1
        Compound: @ 3:11
            Print: @ 4:5
                Constant: string, Hello World! @ 4:11
            Print: @ 5:5
                ExprList: @ 5:11
                    Constant: string, Hello @ 5:11
                    ID: mc @ 5:20
                    Constant: string, . Welcome to MC921 @ 5:24
            Return: @ 6:5

To do this, we just write the code in the expression production:

In [ ]:
    def p_expression(self, p):
        """ expression  : assignment_expression
                        | expression COMMA assignment_expression
        """
        if len(p) == 2:   # single expression
            p[0] = p[1]
        else:
            if not isinstance(p[1], ExprList):
                p[1] = ExprList([p[1]])

            p[1].exprs.append(p[3])
            p[0] = p[1]